In [ ]:
import pandas as pd
import csv
import re
import statsmodels.api as sm


from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/sc_airbnb/{}.csv'

listings_file = file_url.format('listings_details')
reviews_file = file_url.format('reviews_details')
calendar_file = file_url.format('calendar_details')

In [ ]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

sf_file = file_url.format('sf_airbnb/listings_details')
ok_file = file_url.format('ok_airbnb/listings_details')
sc_file = file_url.format('sc_airbnb/listings_details')
scz_file = file_url.format('scz_airbnb/listings_details')

In [ ]:
date_to_parse = ['last_scraped','host_since', 'first_review', 'last_review']

listings_sf = pd.read_csv(sf_file, parse_dates=date_to_parse)
listings_ok = pd.read_csv(ok_file, parse_dates=date_to_parse)
listings_sc = pd.read_csv(sc_file, parse_dates=date_to_parse)
listings_scz = pd.read_csv(scz_file, parse_dates=date_to_parse)

In [ ]:
print(listings_sf.shape,listings_ok.shape,listings_sc.shape,listings_scz.shape,listings_sd.shape,listings_la.shape,)

In [ ]:
# Created keys for future references

frames = [listings_sf, listings_ok, listings_sc, listings_scz]
keys = ['listings_sf', 'listings_ok', 'listings_sc', 'listings_scz']

In [ ]:
df_listings = pd.concat(frames, key=keys)

In [ ]:
df_listings.reset_index()

In [ ]:
df_listings.head()

In [ ]:
# df_listings = pd.read_csv(listings_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])

# print (df_listings.shape)
# df_listings.head()

In [ ]:
df_listings.isnull().sum()

In [ ]:
df_listings.columns

In [ ]:
# Noted >90% null value in 'experience_offered', 'host_acceptance_rate', 'neighbourhood_group_cleansed', 'square_feet'
# Remove 'jurisdiction_names'
# # removed 'neighbourhood',  kept'neighbourhood_cleansed',
# # both city and 'smart location' provides city level information, and 'smart_location can be dropped
# # country code and country can be dropped, as all data located within US
# city is parent of neighbourhood_cleansed

columns_to_keep =[
       'id','host_since','host_response_time', 'host_response_rate',
        'host_is_superhost', 'host_listings_count',
       'host_total_listings_count', 
       'host_has_profile_pic', 'host_identity_verified', 
        'neighbourhood_cleansed',
       'city', 
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type','amenities', 
       'price', 'security_deposit',
       'cleaning_fee', 'guests_included',  'minimum_nights','number_of_reviews',
       'first_review', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',  'jurisdiction_names', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count', 'reviews_per_month']

In [ ]:
# # Noted >90% null value in 'experience_offered', 'host_acceptance_rate', 'neighbourhood_group_cleansed'
# # removed 'neighbourhood',  kept'neighbourhood_cleansed',
# # both city and 'smart location' provides city level information, and 'smart_location can be dropped
# # country code and country can be dropped, as all data located within US
# columns_to_drop =['listing_url', 'scrape_id', 'last_scraped', 'experiences_offered', 
#                   'thumbnail_url', 'medium_url', 'xl_picture_url', 'host_acceptance_rate','street',
#                   'neighbourhood', 
#        'neighbourhood_group_cleansed','zipcode','smart_location','country_code', 
#                   'country', 'square_feet','weekly_price', 'monthly_price','calendar_updated', 
#                   'calendar_last_scraped', 'requires_license', 'license']

In [ ]:
# # 'host_since', 'host_location',

# host_related_info_to_drop = ['host_id', 'host_url', 'host_name', 'host_since',
#                              'host_location', 'host_about', 'host_thumbnail_url',
#                              'host_picture_url', 'host_neighbourhood']

In [ ]:
# # The information is useful for now, but can be dropped during modeling

# extra_info_to_drop = []

In [ ]:
# df_smaller_listings = df_listings.drop (columns_to_drop, axis=1)

In [ ]:
df_smaller_listings = df_listings[columns_to_keep]

In [ ]:
df_smaller_listings

In [ ]:
# This is to onvert strings with percentage sign to float

def percent_to_float(string):
    """
    Convert strings with percentage sign to float
    """
    percentage = float(str(string).strip('%'))/100
    return percentage

In [ ]:
# Convert host_response_rate to float

df_smaller_listings['host_response_rate'] = df_smaller_listings['host_response_rate'].apply(lambda x: percent_to_float(x))

In [ ]:
def currency_to_float(string):
    """
    Convert currency to float
    """
    float_ = float(str(string).strip('$').replace(',',''))
    return float_

In [ ]:
# Convert host_response_rate to float

df_smaller_listings['price'] = df_smaller_listings['price'].apply(lambda x: currency_to_float(x))
df_smaller_listings['security_deposit'] = df_smaller_listings['security_deposit'].apply(lambda x: currency_to_float(x))
df_smaller_listings['cleaning_fee'] = df_smaller_listings['cleaning_fee'].apply(lambda x: currency_to_float(x))

In [ ]:
df_smaller_listings.columns

In [ ]:
df_smaller_listings.head()

In [ ]:
df_smaller_listings.shape

In [ ]:
df_smaller_listings.price.describe(percentiles=[.05, .10, .25, .5, .75, .90, .95])

In [ ]:
# Remove outliers. Use 5% and 95% percentiles as cutoff

df_smaller_listings = df_smaller_listings[(df_smaller_listings['price']<=500) 
                                         & (df_smaller_listings['price']>=40)]

In [ ]:
df_smaller_listings.shape

In [ ]:

# df_amenities = df_smaller_listings.amenities.str.replace("[{}]", "") 
# print(df_amenities

In [ ]:
# print(df_amentities[29])

In [ ]:
# amenity_ohe = df_amenities.str.get_dummies(sep = ",")

In [ ]:
# amenity_ohe

In [ ]:
# drop_for_simple_model=['id', 'name', 'summary', 'space', 'description', 
#                        'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
#                        'house_rules', 'picture_url', 'host_id', 'host_url', 'host_name','host_about',
#                       'host_thumbnail_url', 'host_picture_url', 'latitude', 'longitude',]

In [ ]:
df_smaller_listings['jurisdiction_names'] = df_smaller_listings['jurisdiction_names'].str.replace('[^\w\s,]', '')
df_smaller_listings['amenities'] = df_smaller_listings['amenities'].str.replace('[^\w\s,]', '')

In [ ]:
df_smaller_listings.amenities

In [ ]:
df_amenities = df_smaller_listings.amenities.str.get_dummies(sep = ",")

In [ ]:
df_amenities

In [ ]:
common_amenities=[]

for column in df_amenities.columns:
    value = df_amenities[column].quantile(0.75)
    if value>0:
        common_amenities.append(column)

common_amenities

In [ ]:
df_common_amenities = df_amenities[common_amenities]
df_common_amenities.head()

In [ ]:
df_smaller_listings.shape

In [ ]:
df_simple_model = pd.concat([df_smaller_listings, df_common_amenities], axis=1)

In [ ]:
df_simple_model.shape

In [ ]:
df_simple_model.drop('amenities', axis=1, inplace=True)

In [ ]:
df_simple_model.shape

In [ ]:
df_simple_model.columns

In [ ]:
# df_simple_model['host_since']   = pd.to_datetime(df_smaller_listings['host_since'])
# df_simple_model['first_review'] = pd.to_datetime(df_smaller_listings['first_review'])
# df_simple_model['last_review']  = pd.to_datetime(df_smaller_listings['last_review'])

In [ ]:
def duration(starting_time):
    """
    Calculate duration or number of years between current year and input years
    """
    starting_year = starting_time.year
    current_year = datetime.now().year
    duration = current_year - starting_year + 1
    
    return duration

In [ ]:
# Calculate durations for timestamp columns

df_simple_model['host_duration'] = df_simple_model['host_since'].apply(lambda x: duration(x))
df_simple_model['yrs_since_first_review'] = df_simple_model['first_review'].apply(lambda x: duration(x))

In [ ]:
time_stamps =['host_since', 'first_review']
df_simple_model.drop(time_stamps, axis=1, inplace=True)

In [ ]:
# Replace special characters in columns

# df_simple_model.host_verifications = df_simple_model.host_verifications.str.replace('[^\w\s]', '')
# df_simple_model.jurisdiction_names = df_simple_model.jurisdiction_names.str.replace('[^\w\s]', '')
# df_simple_model.amenities = df_simple_model.amenities.str.replace('[^\w\s]', '')


In [ ]:
df_simple_model.shape

In [ ]:
df_simple_model.columns

In [ ]:
df_simple_model.isnull().sum()

In [ ]:
df_simple_model.isnull().sum()

In [ ]:
# Filled null values with average value

df_simple_model['security_deposit'].fillna(
    value=df_simple_model['security_deposit'].mean(),
    inplace=True)

df_simple_model['cleaning_fee'].fillna(
    value=df_simple_model['cleaning_fee'].mean(),
    inplace=True)

In [ ]:
# Convert security deposit to dummy varialbes
df_simple_model['security_deposit'] = (df_simple_model['security_deposit']>0).astype(int)
# df_simple_model['cleaning_fee'] = (df_simple_model['cleaning_fee']>0).astype(int)

In [ ]:
df_simple_model.describe(include='all')

In [ ]:
df_simple_model.review_scores_rating.describe(include='all')

In [ ]:
df_simple_model.review_scores_accuracy.describe(include='all')

In [ ]:
# Those varialbes have big range. Need to narrow down range to enable better modeling


df_simple_model['cleaning_fee'] = df_simple_model['cleaning_fee'].apply(lambda x: np.log10(x) if x>0 else 0)
df_simple_model['review_scores_rating_log10'] = df_simple_model['review_scores_rating'].apply(lambda x: np.log10(x) if x>0 else 0)
df_simple_model['review_scores_accuracy_log10'] = df_simple_model['review_scores_accuracy'].apply(lambda x: np.log10(x) if x>0 else 0)
df_simple_model['review_scores_checkin_log10'] = df_simple_model['review_scores_checkin'].apply(lambda x: np.log10(x) if x>0 else 0)
df_simple_model['review_scores_cleanliness_log10'] = df_simple_model['review_scores_cleanliness'].apply(lambda x: np.log10(x) if x>0 else 0)
df_simple_model['review_scores_communication_log10'] = df_simple_model['review_scores_communication'].apply(lambda x: np.log10(x) if x>0 else 0)
df_simple_model['review_scores_location_log10'] = df_simple_model['review_scores_location'].apply(lambda x: np.log10(x) if x>0 else 0)
df_simple_model['review_scores_value_log10'] = df_simple_model['review_scores_value'].apply(lambda x: np.log10(x) if x>0 else 0)


In [ ]:
df_simple_model.cleaning_fee.describe(include='all')

In [ ]:
review_scores =['review_scores_rating','review_scores_accuracy',
                'review_scores_checkin','review_scores_cleanliness', 
                'review_scores_communication', 'review_scores_location', 
                'review_scores_value']
df_simple_model.drop(review_scores, axis=1, inplace=True)

In [ ]:
df_simple_model['minimum_nights'] = (df_simple_model['minimum_nights']>1).astype(int)

In [ ]:
df_simple_model.minimum_nights

In [ ]:
df_simple_model.isnull().sum()

In [ ]:
df_simple_model_cleaned = df_simple_model.dropna()

In [ ]:
df_simple_model_cleaned.shape

In [ ]:
df_simple_model_cleaned.isnull().sum()

In [ ]:
df_simple_model_cleaned.columns

In [ ]:
# select non-numeric variables and create dummies
categorical_variables = df_simple_model_cleaned.select_dtypes(include=['object']).columns
df_simple_model_cleaned[categorical_variables].head()

In [ ]:
categorical_variables.shape

In [ ]:
df_simple_model_cleaned.shape

In [ ]:
df_simple_model_cleaned[categorical_variables].describe()

In [ ]:
df_categorical = pd.get_dummies(df_simple_model_cleaned[categorical_variables])
df_categorical

In [ ]:
df_categorical.columns

In [ ]:
df_model = pd.get_dummies(data=df_simple_model_cleaned,columns=categorical_variables)

In [ ]:
df_model.shape

In [ ]:
df_model.columns

In [ ]:
# dummy_variables = pd.get_dummies(df_simple_model_cleaned[categorical_vars])

In [ ]:
# df_model = df_simple_model_cleaned.drop(categorical_vars, axis=1)
# df_model = pd.merge(df_model, dummy_vars, left_index=True, right_index=True)

In [ ]:
# # 'host_verifications','amenities',

# categorical_data = ['host_location', 'host_response_time', 
#        'host_is_superhost', 'host_neighbourhood', 
#        'host_has_profile_pic', 'host_identity_verified',
#        'neighbourhood_cleansed', 'city', 'state', 'market',
#        'is_location_exact', 'property_type', 'room_type',  'bed_type', 'has_availability',
#         'jurisdiction_names',
#        'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
#        'require_guest_profile_picture', 'require_guest_phone_verification',
#        ]

In [ ]:
# df_model = pd.get_dummies(df_model, prefix = categorical_data, columns = categorical_data )

In [ ]:
# Create correlation matrix
corr_matrix = df_model.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

In [ ]:
to_drop

In [ ]:
# Drop features 
df_model.drop(to_drop, axis=1, inplace=True)

In [ ]:
df_model.dropna(inplace=True)

In [ ]:
df_model.columns

In [ ]:
# Store as csv file for feature investigation

csv_path = '/Users/xzhou/github/project_archives/files_airbnb/columns_check.csv'

df_model.to_csv(csv_path)

In [ ]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

pkl_listings = pkl_file.format('all_listings')

df_model.to_pickle(pkl_listings)

In [ ]:
# Perform initial modelings to get a sense of performance, and for future selection

X = df_model.drop(['id','price'], axis=1)
y = df_model['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=42)

In [ ]:
ss = StandardScaler()
X_train_trans = ss.fit_transform(X_train)
X_test_trans = ss.transform (X_test)

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(X_train_trans, y_train)

In [ ]:
lr.score(X_test_trans, y_test)

In [ ]:
y_predict = lr.predict(X_test_trans)

In [ ]:
r2_score(y_predict, y_test)

In [ ]:
mean_squared_error(y_predict, y_test)

In [ ]:
model = sm.OLS(y_train, sm.add_constant(X_train_trans))
fit = model.fit()
fit.summary()

In [ ]:
X_train_trans

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=500, 
                               criterion='mse', 
                               random_state=3, 
                               n_jobs=-1)
rf.fit(X_train_trans, y_train)
y_train_pred = rf.predict(X_train_trans)
y_test_pred = rf.predict(X_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

In [ ]:
feature_labels = X.columns
feature_tuples = zip (feature_labels, rf.feature_importances_)
feature_lists = [list(feature_tuple) for feature_tuple in feature_tuples]

In [ ]:
feature_lists.sort(key=lambda x: abs(x[1]), reverse=True)

In [ ]:
feature_lists

In [ ]:
features_to_keep=[]

for (feature, importance) in feature_lists:
    if importance>=0.003:
        features_to_keep.append(feature)
    else:
        break

features_to_keep

In [ ]:
len(features_to_keep)

In [ ]:
no_features_kept = len(features_to_keep)
features_w_importance = feature_lists[0: no_features_kept]
features_w_importance

In [ ]:
X_updated = df_model[features_to_keep]
y_updated = df_model['price']

In [ ]:
X_updated_train, X_updated_test, y_updated_train, y_updated_test = train_test_split(
    X_updated, y_updated, test_size =0.3, random_state=42)

In [ ]:
ssX2 = StandardScaler()
X_updated_train_trans = ssX2.fit_transform(X_updated_train)
X_updated_test_trans = ssX2.transform (X_updated_test)

In [ ]:
rf_updated = RandomForestRegressor(n_estimators=500, 
                                   criterion='mse', 
                                   random_state=3, 
                                   n_jobs=-1)
rf_updated.fit(X_updated_train_trans, y_updated_train)
y_updated_train_pred = rf_updated.predict(X_updated_train_trans)
y_updated_test_pred = rf_updated.predict(X_updated_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_updated_train, y_updated_train_pred),
        mean_squared_error(y_updated_test, y_updated_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_updated_train, y_updated_train_pred),
        r2_score(y_updated_test, y_updated_test_pred)))

In [ ]:
# Store selected feautre for future use

pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

pkl_listings = pkl_file.format('all_listings')

df_test_model.to_pickle(pkl_listings)